In [7]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record

In [3]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")

In [4]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [5]:
# pre-dedupe arangoDB stuff

# create arangoDB database, or open it if it already exists
try:
    corp_db = conn.createDatabase(name="corp_data")
    # from command line:
    # arangoimport --file '''path to CorpData.csv''' --collection corp_data --create-collection true --type csv --server.database corp_data
    # arangoimport --file '''path to CorpIndividualExport.csv''' --collection corp_individual --create-collection true --type csv --server.database corp_data

except:
    corp_db = conn["corp_data"]

In [ ]:
# corp_db["corp_data"]: arangoDB collection
# corp_db["corp_data"][17777999]: arangoDB document (one piece of data, or one corporation)

In [6]:
# extracting the values of the corp_data collection
val_aql = "FOR x IN corp_data RETURN x" # here, corp_data is the name of the collection, not the database
value_query_result = corp_db.AQLQuery(val_aql,rawResults=True, batchSize = 1000)
col_value = {}
ind_val = 0

for value in value_query_result:
    col_value[ind_val] = value
    ind_val += 1

In [7]:
print(col_value[2])

{'_key': '17778001', '_id': 'corp_data/17778001', '_rev': '_eU6dnzS--A', 'DataID': '00n9yj', 'FEIN': '001151215', 'TempFEIN': '', 'EntityName': 'GREENLINK FINANCIAL, LLC', 'EntityTypeDescriptor': 'Foreign Limited Liability Company (LLC)', 'Addr1': '3 MACARTHUR PLACE', 'Addr2': 'SUITE #1000', 'City': 'SANTA ANA', 'State': 'CA', 'PostalCode': '92707', 'CountryCode': 'USA', 'AgentName': 'COGENCY GLOBAL INC.', 'AgentAddr1': '45 SCHOOL STREET', 'AgentAddr2': 'STE 202', 'AgentCity': 'BOSTON', 'AgentState': 'MA', 'AgentPostalCode': '02108', 'DoingBusinessAs': '', 'ForeignName': '', 'JurisdictionState': 'NV', 'JurisdictionCountry': 'USA', 'JurisdictionDate': '02/27/2014', 'DateOfOrganization': '11/03/2014', 'ActiveFlag': 'N', 'InactiveDate': '2020-01-17 00:00:00', 'InactiveType': 'W', 'RevivalDate': '', 'LastDateCertain': '', 'FiscalMonth': '  ', 'FiscalDay': '  ', 'MergerAllowedFlag': '', 'AnnualRptReqFlag': 'Y', 'CorpPublicFlag': '', 'ProfitFlag': 'N', 'ConsentFlag': 'N', 'PartnershipFlag': 

In [8]:
# create dataframe from dictionary of dictionaries
corp_df = pd.DataFrame.from_dict(data = col_value, orient = 'index')

In [16]:
print(corp_df)

             _key                 _id         _rev  DataID       FEIN  \
0        17777999  corp_data/17777999  _eU6dnzS---  00wepr  001519497   
1        17778000  corp_data/17778000  _eU6dnzS--_  00v4bw  001470249   
2        17778001  corp_data/17778001  _eU6dnzS--A  00n9yj  001151215   
3        17778002  corp_data/17778002  _eU6dnzS--B  00l0iw  001079650   
4        17778003  corp_data/17778003  _eU6dnzS--C  000002  001052715   
...           ...                 ...          ...     ...        ...   
1369463  19147517  corp_data/19147517  _eU6dtH6--X  00vrf1  001494237   
1369464  19147518  corp_data/19147518  _eU6dtH6--Y  00slfo  001374222   
1369465  19147519  corp_data/19147519  _eU6dtH6--Z  00tltb  001411442   
1369466  19147520  corp_data/19147520  _eU6dtH6--a  00kdn6  043125411   
1369467  19147521  corp_data/19147521  _eU6dtH6--b  00wkuw  001525489   

        TempFEIN                         EntityName  \
0                                  MILLER PLACE LLC   
1            

In [10]:
# concatenate entity address
corp_df.loc[:,'EntityAddr'] = [', '.join((str(a),str(b),str(c),str(d))) for a,b,c,d in zip(corp_df['Addr1'],corp_df['Addr2'],corp_df['City'],corp_df['State'])]
# double comma problem
# append zip to address with no comma
corp_df.loc[:,'EntityAddr'] = [' '.join((str(a),str(b))) for a,b in zip(corp_df['EntityAddr'],corp_df['PostalCode'])]
corp_df.loc[:,'EntityAddr'] = corp_df.EntityAddr.str.strip()

# concatenate agent address
corp_df.loc[:,'AgentAddr'] = [', '.join((str(a),str(b),str(c),str(d))) for a,b,c,d in zip(corp_df['AgentAddr1'],corp_df['AgentAddr2'],corp_df['AgentCity'],corp_df['AgentState'])]
# append zip to agent address
corp_df.loc[:,'AgentAddr'] = [' '.join((str(a),str(b))) for a,b in zip(corp_df['AgentAddr'],corp_df['AgentPostalCode'])]
corp_df.loc[:,'AgentAddr'] = corp_df.AgentAddr.str.strip()

# slice
corp_df_sliced = corp_df[['_key','_id','EntityName','EntityAddr','AgentName','AgentAddr']]
print(corp_df_sliced)

             _key                 _id                         EntityName  \
0        17777999  corp_data/17777999                   MILLER PLACE LLC   
1        17778000  corp_data/17778000                 A & V BUILDERS LLC   
2        17778001  corp_data/17778001           GREENLINK FINANCIAL, LLC   
3        17778002  corp_data/17778002                           DTN, LLC   
4        17778003  corp_data/17778003      JOSH SHAPIRO SPORTS CAMPS LLC   
...           ...                 ...                                ...   
1369463  19147517  corp_data/19147517                         NANYDA LLC   
1369464  19147518  corp_data/19147518                  LITTLE PECAN, LLC   
1369465  19147519  corp_data/19147519                     WP MOBILE INC.   
1369466  19147520  corp_data/19147520  PATTERSON CLEANING SERVICES, INC.   
1369467  19147521  corp_data/19147521                         GOBLIT LLC   

                                                EntityAddr  \
0        189 WELLS AVENUE

In [11]:
# extracting the values of the corp_individual collection
indiv_aql = "FOR x IN corp_individual RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1
    
print(col_indiv[0])

{'_key': '12872246', '_id': 'corp_individual/12872246', '_rev': '_eU6cXum---', 'DataID': '007gxq', 'IndividualTitle': 'TREASURER', 'IndividualTypeFlag': 'B', 'FirstName': 'CHARLES', 'LastName': 'BURKE', 'MiddleName': '', 'Suffix': '', 'TermExpiration': '', 'BusAddr1': '55 CROSS LANE, ', 'BusCity': 'BEVERLY', 'BusState': 'MA', 'BusCountryCode': 'USA', 'BusPostalCode': '', 'ResAddr1': '55 CROSS LANE, ', 'ResCity': 'BEVERLY', 'ResState': 'MA', 'ResCountryCode': 'USA', 'ResPostalCode': ''}


In [12]:
indiv_df = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')

In [14]:
# concatenate business address
indiv_df.loc[:,'BusAddr'] = [', '.join((str(a),str(b),str(c))) for a,b,c in zip(indiv_df['BusAddr1'],indiv_df['BusCity'],indiv_df['BusState'])]
# append zip to address with no comma
indiv_df.loc[:,'BusAddr'] = [' '.join((str(a),str(b))) for a,b in zip(indiv_df['BusAddr'],indiv_df['BusPostalCode'])]
indiv_df.loc[:,'BusAddr'] = indiv_df.BusAddr.str.strip()

# concatenate residential address
indiv_df.loc[:,'ResAddr'] = [', '.join((str(a),str(b),str(c))) for a,b,c in zip(indiv_df['ResAddr1'],indiv_df['ResCity'],indiv_df['ResState'])]
# append zip to address with no comma
indiv_df.loc[:,'ResAddr'] = [' '.join((str(a),str(b))) for a,b in zip(indiv_df['ResAddr'],indiv_df['ResPostalCode'])]
indiv_df.loc[:,'ResAddr'] = indiv_df.ResAddr.str.strip()

# concatenate name
indiv_df.loc[:,'IndividualName'] = [' '.join((str(a),str(b),str(c))) for a,b,c in zip(indiv_df['FirstName'],indiv_df['MiddleName'],indiv_df['LastName'])]

# slice
indiv_df_sliced = indiv_df[['_key','_id','IndividualName','BusAddr','ResAddr']]
print(indiv_df_sliced)

             _key                       _id             IndividualName  \
0        12872246  corp_individual/12872246             CHARLES  BURKE   
1        12872247  corp_individual/12872247       GERALDINE M. SHAHEEN   
2        12872248  corp_individual/12872248  ALEXSANDRO OLIVEIRA VIANA   
3        12872249  corp_individual/12872249          KEVIN LEO BUCKLEY   
4        12872250  corp_individual/12872250   NIVALDO LEMOS NASCIMENTO   
...           ...                       ...                        ...   
4905634  17777964  corp_individual/17777964       CHRISTOPHER  SPENCER   
4905635  17777965  corp_individual/17777965      LAWRENCE M. PETTORUTO   
4905636  17777966  corp_individual/17777966      LAWRENCE M. PETTORUTO   
4905637  17777967  corp_individual/17777967      LAWRENCE M. PETTORUTO   
4905638  17777968  corp_individual/17777968      LAWRENCE M. PETTORUTO   

                                              BusAddr  \
0                        55 CROSS LANE, , BEVERLY, MA 

In [ ]:
#*****EXPERIMENTAL, DON'T RUN*****

# Parcel data from MassGIS Data
# https://www.mass.gov/forms/massgis-request-statewide-parcel-data

# Residential Land Use Codes from MA Dept of Revenue
# https://www.mass.gov/files/documents/2016/08/wr/classificationcodebook.pdf
# Codes are 101*-109*, 031*, and 013*
# Often include suffixes (letters, zeroes or no character), thus regex *?

USE_CODES = '^1[0-1][1-9]*?|^013*?|^031*?'
# BOS_CODES = '^R[1-4]$|^RC$|^RL$|^CD$|^A$'

# medparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

def read_res(file_dict):
    df = pd.DataFrame()
    for town, file, in file_dict.items():
        town_df = gpd.read_file(file).drop('geometry', axis='columns')
        town_df['town'] = town
        df = df.append(town_df, ignore_index=True)
    return df

In [33]:
#*****EXPERIMENTAL, DON'T RUN*****
ADDRESS_MAP = {
    'Recipient': 'office',
    'AddressNumber': 'add1',
    'AddressNumberPrefix': 'add',
    'AddressNumberSuffix': 'add1',
    'StreetName': 'add1',
    'StreetNamePreDirectional': 'add1',
    'StreetNamePreModifier': 'add1',
    'StreetNamePreType': 'add1',
    'StreetNamePostDirectional': 'add1',
    'StreetNamePostModifier': 'add1',
    'StreetNamePostType': 'add1',
    'CornerOf': 'add1',
    'IntersectionSeparator': 'add1',
    'LandmarkName': 'add1',
    'USPSBoxGroupID': 'add1',
    'USPSBoxGroupType': 'add1',
    'USPSBoxID': 'add1',
    'USPSBoxType': 'add1',
    'BuildingName': 'office',
    'OccupancyType': 'add2',
    'OccupancyIdentifier': 'add2',
    'SubaddressIdentifier': 'add2',
    'SubaddressType': 'add2',
    'PlaceName': 'city',
    'StateName': 'state',
    'ZipCode': 'zip',
    'PostCode': 'zip',
    'PostalCode': 'zip',
}

def prompt_or_not(parse_dict, type, label = None):
    if label:
        t = label
    else:
        t = type
    p = input(f"{t}? (If none, hit 'Enter'.): ")
    if len(p) > 0:
        parse_dict[type] = p
    return parse_dict

def dict_to_series(a:dict, b:pd.Series, label):
    if label in a.keys():
        b[label] = a[label]
    else:
        b[label] = None
    return b

def parse_address(add, full = False):
    """Parses addresses. Use full when full mailing address present."""

    components = [
            'add1',
            'add2',
    ]
    if full:
        components = components + [
            'city',
            'state',
            'zip',
            'office'
        ]
    a = usaddress.tag(add, tag_mapping=ADDRESS_MAP)[0]

    s = pd.Series(dtype = 'object')
    for c in components:
        s = dict_to_series(a, s, c)
    return s

result = parse_address('157 LELAND ST # 2, FRAMINGHAM, MA 01702',full=True)
print(result)

add1      157 LELAND ST
add2                # 2
city         FRAMINGHAM
state                MA
zip               01702
office             None
dtype: object


In [31]:
print(normalize_address_record('5 EXCHANGE STREET, SUITE 4, MILFORD, MA 01757'))
print(normalize_address_record('73 OXFORD ST., , WINCHESTER, MA 01890'))
print(normalize_address_record('67 ELM HILL AVENUE APT. 7,JAMAICA PLAIN, MA'))

{'address_line_1': '5 EXCHANGE ST', 'address_line_2': 'STE 4', 'city': 'MILFORD', 'state': 'MA', 'postal_code': '01757'}
{'address_line_1': '73 OXFORD ST', 'address_line_2': None, 'city': 'WINCHESTER', 'state': 'MA', 'postal_code': '01890'}
{'address_line_1': '67 ELM HILL AVE', 'address_line_2': 'APT 7', 'city': 'JAMAICA PLAIN', 'state': 'MA', 'postal_code': None}


In [ ]:
#*****EXPERIMENTAL, DON'T RUN*****
#QGIS - python API for CUSTOM QGIS APPLICATION

from qgis.core import *

# Supply path to qgis install location
QgsApplication.setPrefixPath("/path/to/qgis/installation", True)

# Create a reference to the QgsApplication.  Setting the
# second argument to False disables the GUI.
qgs = QgsApplication([], False)

# Load providers
qgs.initQgis()

# Write your code here to load some layers, use processing
# algorithms, etc.

# Finally, exitQgis() is called to remove the
# provider and layer registries from memory
qgs.exitQgis()

In [ ]:
#process geodatabase
data = gpd.read_file("../MassGIS_L3_Parcels_gdb/MassGIS_L3_Parcels.gdb", driver='FileGDB', layer=2)
print(data)